In [81]:
import pandas as pd
import numpy as np
from datetime import datetime
from PIL import Image
from sklearn.linear_model import LinearRegression
import warnings

warnings.filterwarnings('ignore')

Step 1. For each month (image), count the number of pixels above a certain light threshold (lower_bound).

In [82]:
#Set up function to count pixels in image that meet criteria
def count_color_pixels_in_range(name, lower_bound, upper_bound):

    image_path = f'Permian\\{name}.png'
    
    img = Image.open(image_path)
    
    img = img.convert("L")
    
    pixels = img.getdata()
    
    color_count = 0

    for pixel in pixels:
        if lower_bound[0] <= pixel <= upper_bound[0]:
            color_count += 1

    return color_count

In [121]:
lower_bound = (225,)  # Minimum grayscale value level to return as "light" - edit as necessary
upper_bound = (255,)  # Maximum value - do not edit

In [122]:
#Set up list of file names to retrieve based on dates
names_list = []
for year in range(2013,2024):
    for month in range(1,13):
        if month < 10:
            month_str = f'0{month}'
        else:
            month_str = str(month)
        name = f'{year}-{month_str}'
        names_list.append(name)

In [123]:
#Set up, append to list light count in each image
lights_count_list = []

for name in names_list:
    lights_count_list.append(count_color_pixels_in_range(name,lower_bound, upper_bound))

In [124]:
#Create dates list
date_list = []
for x in names_list:
    year = int(x[0:4])
    month = int(x[5:7])
    date = datetime(year, month, 1).date()
    date_list.append(date)


#Create datafarme of light output at each month
light_output_df = pd.DataFrame({
    'Date': date_list,
    'Light Output (Pixels)': lights_count_list
})

light_output_df

,Date,Light Output (Pixels)
0,2013-01-01,39631
1,2013-02-01,38303
2,2013-03-01,43867
3,2013-04-01,46474
4,2013-05-01,44879
...,...,...
127,2023-08-01,209331
128,2023-09-01,195789
129,2023-10-01,208211
130,2023-11-01,200613


Step 2. Feed in actual oil output figures from EIA data set, set training window of n periods to fit regression, test OOS performance


In [126]:
#Retrieve, clean EIA oil output figures
oil_output_df = pd.read_excel('EIA Figure - Texas Million Barrels.xlsx',parse_dates=['Month'])
oil_output_df.rename(columns={'Month': 'Date'},inplace=True)
oil_output_df['Date'] = [x.date() for x in oil_output_df['Date']]

#Merge EIA figures with light figures
matched_data = pd.merge(light_output_df, oil_output_df, on='Date', how='left')
matched_data.set_index('Date',inplace=True)

#Set up sum-squared-residuals, sum-squared-total lists for rsq calculation
ssr_list = []
sst_list = []

#Set training window size - edit as necessary
windowsize = 24

#Roll training window
for t in range(0,len(matched_data.index)-windowsize):
    start_date = matched_data.index[t]
    end_date = matched_data.index[t+windowsize-1]
    oos_date = matched_data.index[t+windowsize]

    #Get training window via date indexers
    training_window = matched_data.loc[start_date:end_date]

    #Fit model in-sample
    y = training_window['EIA Figure']
    X = training_window[['Light Output (Pixels)']]
    regression = LinearRegression().fit(X,y)

    #Retrieve regression coefficients
    β0 = regression.intercept_
    β1 = regression.coef_[0]
    
    #Test OOS - one month after training window_end
    OOS_y = matched_data.loc[oos_date]['EIA Figure']
    OOS_X = matched_data.loc[oos_date]['Light Output (Pixels)']

    y_predicted = β0 + β1 * OOS_X
    residual_squared = (y_predicted - OOS_y)**2 
    y_var_squared = (OOS_y - np.average(y))**2

    ssr_list.append(residual_squared)
    sst_list.append(y_var_squared)

oos_rsq = 1 - (sum(ssr_list)/sum(sst_list))

print(f'Out-of-Sample r-squared: {oos_rsq}')

Out-of-Sample r-squared: 0.5125994866591173
